In [1]:
import numpy as np
import argparse

In [6]:
# Initialize model parameters
M = 64 # args.M
N = 64 # args.N
M_LEN = M + 1
N_LEN = N + 1
L_OUT = 4000 # args.L_OUT
ITMAX = 4000
dt = 90.
tdt = dt
dx = 100000.
dy = 100000.
fsdx = 4. / (dx)
fsdy = 4. / (dy)
a = 1000000.
alpha = 0.001
el = N * dx
pi = 4. * np.arctan(1.)
tpi = 2. * pi
di = tpi / M
dj = tpi / N
dtheta = tpi / M
pcf = (pi * pi * a * a) / (el * el)
SIZE = M_LEN * N_LEN

    

In [7]:
# Model Variables
u = np.zeros((M_LEN, N_LEN))
v = np.zeros((M_LEN, N_LEN))
p = np.zeros((M_LEN, N_LEN))
unew = np.zeros((M_LEN, N_LEN))
vnew = np.zeros((M_LEN, N_LEN))
pnew = np.zeros((M_LEN, N_LEN))
uold = np.zeros((M_LEN, N_LEN))
vold = np.zeros((M_LEN, N_LEN))
pold = np.zeros((M_LEN, N_LEN))
cu = np.zeros((M_LEN, N_LEN))
cv = np.zeros((M_LEN, N_LEN))
z = np.zeros((M_LEN, N_LEN))
h = np.zeros((M_LEN, N_LEN))
psi = np.zeros((M_LEN, N_LEN))
    
# Initial values of the stream function and p
for i in range(M + 1):
    for j in range(N + 1):
        psi[i, j] = a * np.sin((i + .5) * di) * np.sin((j + .5) * dtheta)
        p[i, j] = pcf * (np.cos(2. * (i) * di) + np.cos(2. * (j) * dtheta)) + 50000.
            
# Calculate initial u and v
for i in range(M):
    for j in range(N):
        u[i + 1, j] = (psi[i + 1, j + 1] - psi[i + 1, j]) / (dy)
        v[i, j + 1] = (-psi[i, j + 1] + psi[i + 1, j + 1]) / (dx)
            
# # Periodic Boundary conditions
for j in range(N):
    u[0,j] = u[M, j]
    v[M, j + 1] = v[0, j + 1]
        
for i in range(M):
    u[i + 1, N] = u[i + 1, 0]
    v[i, 0] = v[i, N]

u[0, 0] = u[0, N]
v[M, 0] = v[0, 0]
    
# Save initial conditions
uold = np.copy(u)
vold = np.copy(v)
pold = np.copy(p)

 

In [8]:
# Print initial conditions
if L_OUT:
    print(" Number of points in the x direction: ", M)
    print(" Number of points in the y direction: ", N)
    print(" grid spacing in the x direction: ", dx)
    print(" grid spacing in the y direction: ", dy)
    print(" time step: ", dt)
    print(" time filter coefficient: ", alpha)
    print(" Initial u:\n", u[0:min(M, N), 0:min(M, N)])
    print(" Initial v:\n", v[0:min(M, N), 0:min(M, N)])
    print(" Initial p:\n", p[0:min(M, N), 0:min(M, N)])
        


 Number of points in the x direction:  64
 Number of points in the y direction:  64
 grid spacing in the x direction:  100000.0
 grid spacing in the y direction:  100000.0
 time step:  90.0
 time filter coefficient:  0.001
 Initial u:
 [[ 0.          0.04722749  0.04607929 ...  0.04722749  0.04792086
   0.04815273]
 [ 0.14330109  0.14122765  0.13779411 ...  0.14122765  0.14330109
   0.14399446]
 [ 0.23730125  0.23386771  0.22818189 ...  0.23386771  0.23730125
   0.23844945]
 ...
 [-0.23730125 -0.23386771 -0.22818189 ... -0.23386771 -0.23730125
  -0.23844945]
 [-0.14330109 -0.14122765 -0.13779411 ... -0.14122765 -0.14330109
  -0.14399446]
 [-0.04792086 -0.04722749 -0.04607929 ... -0.04722749 -0.04792086
  -0.04815273]]
 Initial v:
 [[ 0.04792086  0.14330109  0.23730125 ... -0.23730125 -0.14330109
  -0.04792086]
 [ 0.04722749  0.14122765  0.23386771 ... -0.23386771 -0.14122765
  -0.04722749]
 [ 0.04607929  0.13779411  0.22818189 ... -0.22818189 -0.13779411
  -0.04607929]
 ...
 [ 0.047227

In [9]:
# Main time loop
for ncycle in range(ITMAX):
    if ncycle % 100 == 0:
        print("cycle number ", ncycle)
    # Calculate cu, cv, z, and h
    for i in range(M):
        for j in range(N):
            cu[i + 1, j] = .5 * (p[i + 1, j] + p[i, j]) * u[i + 1, j]
            cv[i, j + 1] = .5 * (p[i, j + 1] + p[i, j]) * v[i, j + 1]
            z[i + 1, j + 1] = (fsdx * (v[i + 1, j + 1] - v[i, j + 1]) -
                                fsdy * (u[i + 1, j + 1] - u[i, j + 1] )
                                ) / (p[i, j] + p[i + 1, j] + p[i + 1, j + 1] + p[i, j + 1])
            h[i, j] = p[i, j] + 0.25 * (u[i + 1, j] * u[i + 1, j] + u[i, j] * u[i, j] +
                                        v[i, j + 1] * v[i, j + 1] + v[i, j] * v[i, j])
        
    # # Periodic Boundary conditions
    for j in range(N):
        cu[0, j] = cu[M, j]
        cv[M, j + 1] = cv[0, j + 1]
        z[0,j + 1] = z[M, j + 1]
        h[M, j] = h[0, j]

    for i in range(M):
        cu[i + 1, N] = cu[i + 1, 0]
        cv[i, 0] = cv[i, N]
        z[i + 1, N] = z[i + 1, 0]
        h[i, N] = h[i, 0]

    cu[0, 0] = cu[0, N]
    cv[M, 0] = cv[0, 0]
    z[0, 0] = z[M, N]
    h[M, N] = h[0, 0]
        
    # Calclulate new values of u,v, and p
    tdts8 = tdt / 8.
    tdtsdx = tdt / dx
    tdtsdy = tdt / dy
        
    for i in range(M):
        for j in range(N):
            unew[i + 1, j] = (uold[i + 1, j] + tdts8 * (z[i + 1, j + 1] + z[i + 1, j]) *
                                (cv[i + 1, j + 1] + cv[i + 1, j] + cv[i, j + 1] + cv[i, j]) -
                                tdtsdx * (h[i + 1, j] - h[i, j])
                                )
            vnew[i, j + 1] = (vold[i, j + 1] - tdts8 * (z[i + 1, j + 1] + z[i, j + 1]) *
                                (cu[i + 1, j + 1] + cu[i + 1, j] + cu[i, j + 1] + cu[i, j]) -
                                tdtsdy * (h[i, j + 1] - h[i, j])
                                )
            pnew[i, j] = (pold[i, j] - tdtsdx * (cu[i + 1, j] - cu[i, j]) -
                                tdtsdy * (cv[i, j + 1] - cv[i, j])
                                )
                
    # Periodic Boundary conditions
    for j in range(N):
        unew[0, j] = unew[M, j]
        vnew[M, j + 1] = vnew[0, j + 1]
        pnew[M, j] = pnew[0, j]

    for i in range(M):
        unew[i + 1, N] = unew[i + 1, 0]
        vnew[i, 0] = vnew[i, N]
        pnew[i, N] = pnew[i, 0]

    unew[0, 0] = unew[0, N]
    vnew[M, 0] = vnew[0, 0]
    pnew[0, 0] = pnew[0, 0]
        
   

cycle number  0
cycle number  100
cycle number  200
cycle number  300
cycle number  400
cycle number  500
cycle number  600
cycle number  700
cycle number  800
cycle number  900
cycle number  1000
cycle number  1100
cycle number  1200
cycle number  1300
cycle number  1400
cycle number  1500
cycle number  1600
cycle number  1700
cycle number  1800
cycle number  1900
cycle number  2000
cycle number  2100
cycle number  2200
cycle number  2300
cycle number  2400
cycle number  2500
cycle number  2600
cycle number  2700
cycle number  2800
cycle number  2900
cycle number  3000
cycle number  3100
cycle number  3200
cycle number  3300
cycle number  3400
cycle number  3500
cycle number  3600
cycle number  3700
cycle number  3800
cycle number  3900


In [10]:
 # Print initial conditions
if L_OUT:
    print("cycle number ", ITMAX)
    print(" diagonal elements of u:\n", unew[0:min(M, N), 0:min(M, N)])
    print(" diagonal elements of v:\n", vnew[0:min(M, N), 0:min(M, N)])
    print(" diagonal elements of p:\n", pnew[0:min(M, N), 0:min(M, N)])

cycle number  4000
 diagonal elements of u:
 [[ 0.          0.04720333  0.0460482  ...  0.04723568  0.04792089
   0.04814443]
 [ 0.14328863  0.14120425  0.13776426 ...  0.14123614  0.1433013
   0.14398639]
 [ 0.23729007  0.23384567  0.22815399 ...  0.23387649  0.23730179
   0.23844191]
 ...
 [-0.23731236 -0.23389044 -0.22821216 ... -0.23386037 -0.23730083
  -0.23845654]
 [-0.14331299 -0.14125147 -0.13782541 ... -0.14122005 -0.14330096
  -0.14400226]
 [-0.04793318 -0.0472518  -0.04611087 ... -0.0472196  -0.04792086
  -0.04816094]]
 diagonal elements of v:
 [[ 0.04791679  0.14329987  0.23730199 ... -0.23730102 -0.14330107
  -0.0479209 ]
 [ 0.04721951  0.14123684  0.23387763 ... -0.23386151 -0.14122075
  -0.04721984]
 [ 0.04607169  0.13781229  0.22820155 ... -0.22817132 -0.13778152
  -0.04606471]
 ...
 [ 0.04721951  0.14120542  0.23384756 ... -0.23389233 -0.14125264
  -0.04725219]
 [ 0.04791264  0.1432854   0.23728666 ... -0.23731673 -0.14331733
  -0.0479374 ]
 [ 0.04814443  0.14398639  0